In [1]:
# Kullandığımız kütüphaneleri import ettik
import tkinter as tk
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re

import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import scipy.stats as stats

from sklearn import linear_model
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoCV, RidgeCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import r2_score


#Jupyter kernelda veri görmek için ayarlamalar
pd.set_option('display.max_columns', 75)
pd.set_option('display.max_rows', 100)

In [2]:
#bu kısımda siteden futbolcu verilerini çekmek için kullandık,bunun için Python BeatifulSoup kütüphanesini kullandık.Futbolcu bilgilerinin hepsini çektik ancak kullanmadığımızda oldu
player_list = []#futbolcu listesi
i = 0
while i < 6000:#6000 tane futbolcu çektik
    url_list = "https://sofifa.com/?&showCol%5B%5D=pi&showCol%5B%5D=ae&showCol%5B%5D=hi&showCol%5B%5D=wi&showCol%5B%5D=pf&showCol%5B%5D=oa&showCol%5B%5D=pt&showCol%5B%5D=bo&showCol%5B%5D=bp&showCol%5B%5D=gu&showCol%5B%5D=vl&showCol%5B%5D=wg&showCol%5B%5D=rc&showCol%5B%5D=ta&showCol%5B%5D=cr&showCol%5B%5D=fi&showCol%5B%5D=he&showCol%5B%5D=sh&showCol%5B%5D=vo&showCol%5B%5D=ts&showCol%5B%5D=dr&showCol%5B%5D=cu&showCol%5B%5D=fr&showCol%5B%5D=lo&showCol%5B%5D=bl&showCol%5B%5D=to&showCol%5B%5D=ac&showCol%5B%5D=sp&showCol%5B%5D=ag&showCol%5B%5D=re&showCol%5B%5D=ba&showCol%5B%5D=tp&showCol%5B%5D=so&showCol%5B%5D=ju&showCol%5B%5D=st&showCol%5B%5D=sr&showCol%5B%5D=ln&showCol%5B%5D=te&showCol%5B%5D=ar&showCol%5B%5D=in&showCol%5B%5D=po&showCol%5B%5D=vi&showCol%5B%5D=pe&showCol%5B%5D=cm&showCol%5B%5D=td&showCol%5B%5D=ma&showCol%5B%5D=sa&showCol%5B%5D=sl&showCol%5B%5D=tg&showCol%5B%5D=gd&showCol%5B%5D=gh&showCol%5B%5D=gc&showCol%5B%5D=gp&showCol%5B%5D=gr&showCol%5B%5D=tt&showCol%5B%5D=bs&showCol%5B%5D=ir&showCol%5B%5D=pac&showCol%5B%5D=sho&showCol%5B%5D=pas&showCol%5B%5D=dri&showCol%5B%5D=def&showCol%5B%5D=phy&offset="+str(i)
    response = requests.get(url_list)#verilen url döndürdüğü Http response=200
    page = response.text#response ile alınan sayfanın bilgilerin içeriğini alma
    soup = BeautifulSoup(page,"lxml")#sayfa bilgileri html ve xml formatına dönüştürme
    rows = soup.find_all("tr")#sayfadaki tüm sutün alma
    for row in rows:
        cells = row.find_all('td')#sayfadaki tüm satırları stringe çevirip regular expression kullanarak dict e çevirme
        cells_to_string = str(cells)
        cells_sub = (re.sub(re.compile('<.*?>'),'',cells_to_string))
        player_list.append(cells_sub)
        result = pd.DataFrame(player_list)#dataframe ekleme
    i += 60#indis arttırma sayfa boyutu 60 aldığı için 60 kullanıldı

In [3]:
result.drop_duplicates(subset=None, keep="first", inplace=True)#gereksiz bazı satır ve sutünları kaldırdık

In [4]:
result.to_pickle("PlayerList1")#oluşturduğumuz dataframe tekrar tekrar kullanmak için pickle dosyasına kaydetip tekrar çalıştırma

In [5]:
result = pd.read_pickle("PlayerList1")

In [6]:
table_players = soup.find("table",attrs={"class":"table table-hover persist-area"})#sitedeki tablodan sütun adı çekmek için kullandık ve ekledik
header_players = table_players.find_all("th")
column_name_players = []
for row in header_players:
    column_name_players.append(row.text)

In [7]:
#boşluk ve bazı yanlış sütunları kaldırmak için yeni sutün
new_column_name_players = [
 'Name',
 'Age',
 'Overall',
 'Potential',
 'Team',
 'Height',
 'Weight',
 'Foot',
 'Best_Overall',
 'Position',
 'Growth',
 'Value',
 'Wage',
 'Release_Clause',
 'Attacking',
 'Crossing',
 'Finishing',
 'Heading_Accuracy',
 'Short_Passing',
 'Volleys',
 'Skill',
 'Dribbling',
 'Curve',
 'FK_Accuracy',
 'Long_Passing',
 'Ball_Control',
 'Movement',
 'Acceleration',
 'Sprint_Speed',
 'Agility',
 'Reactions',
 'Balance',
 'Power',
 'Shot_Power',
 'Jumping',
 'Stamina',
 'Strength',
 'Long_Shots',
 'Mentality',
 'Aggression',
 'Interceptions',
 'Positioning',
 'Vision',
 'Penalties',
 'Composure',
 'Defending',
 'Marking',
 'Standing_Tackle',
 'Sliding_Tackle',
 'Goalkeeping',
 'GK_Diving',
 'GK_Handling',
 'GK_Kicking',
 'GK_Positioning',
 'GK_Reflexes',
 'Total_Stats',
 'Base_Stats',
 'Int_Reputation',
 'PAC',
 'SHO',
 'PAS',
 'DRI',
 'DEF',
 'PHY']

In [8]:
#Verileri \n , "" ayırmak için yazılan kod
result = pd.read_pickle("PlayerList1")
result = result[0].str.split(',',expand=True)# , ayırma
result[1] = result[1].str.replace("\n ","")# \n silme
#bilesik mevkileri ayırma
chars_to_remove = ["GK","RB","RCB","CB","LCB","LB","RWB","RDM","CDM","LDM","LWB","RM","RCM","CM","LCM","LM","LAM","CAM","RAM","RW","RF","CF","LF","LW","LS","ST","RS"]
for char in chars_to_remove:
    result[1]=result[1].str.replace(char,"")
    result[1]=result[1].str.lstrip()

# bileşik \n leri silme
result[5] = result[5].str.replace("\n\n\n\n","")
result[5] = result[5].str.split("\n").str[0]
result[5] = result[5].str.strip()

#kiralık oyuncuların kiralık bilgilerini düşürme
result[6] = result[6].replace(r'Loan', np.nan, regex=True)
result = result.iloc[1:]
result_WoLoan = result[result[6].notna()]
result_WoLoan.drop(result_WoLoan.columns[[0,6,66,67,68,69]],axis=1, inplace=True)
mask_loan = result[6].isnull()
result_loan = result.loc[mask_loan].shift(-1, axis=1)
result_loan.drop_duplicates(subset=[6], keep="first", inplace=True)
result_loan.drop(result_loan.columns[[5,6,66,67,68,69]],axis=1, inplace=True)
result_WoLoan.columns = new_column_name_players
result_loan.columns = new_column_name_players
result_master = pd.concat([result_WoLoan,result_loan]).reset_index()

del result_master["index"]
result_master.drop_duplicates(subset=None, keep="first",inplace=True)
result_master = result_master[~result_master["Age"].str.contains("\n")]

TypeError: 'DataFrame' object is not callable

In [12]:
result_master = result_master.drop(198, axis=0)#hatalı sütunları silinmesi indexler değişebilir
result_master = result_master.drop(1902, axis=0)
result_master = result_master.drop(3699, axis=0)
result_master = result_master.drop(5308, axis=0)
result_master = result_master.drop(884, axis=0)

In [13]:
#€ ve M,K para birimleri SİLME 1M=1 1K=0.001
result_master["Value"] = result_master["Value"].str.replace("€","")
result_master["Value"] = result_master["Value"].str.replace("M","")
result_master.loc[result_master["Value"].str.contains("K"),"Value"]=result_master["Value"].str.split("K").str[0].astype(float)/1000
result_master["Value"] = round(result_master["Value"].astype(float),2)
result_master = result_master.loc[result_master["Value"]>0]
# Maaş sütunun çevirme
result_master["Wage"] = result_master["Wage"].str.replace("€","")
result_master.loc[result_master["Wage"].str.contains("K"),"Wage"]=result_master["Wage"].str.split("K").str[0].astype(float)*1000
result_master["Wage"] = round(result_master["Wage"].astype(float)/1000000,5)
#Serbest Kalma Maddesini sayıya çevirme
result_master["Release_Clause"] = result_master["Release_Clause"].str.replace("€","")
result_master["Release_Clause"] = result_master["Release_Clause"].str.replace("M","")
result_master.loc[result_master["Release_Clause"].str.contains("K"),"Release_Clause"]=result_master["Release_Clause"].str.split("K").str[0].astype(float)/1000
result_master["Release_Clause"] = round(result_master["Release_Clause"].astype(float),2)
#Değeri olmayan (Free Agent) oyuncuları silme(Ronaldinho, Juninho, Xavi etc)
result_master = result_master.loc[result_master["Composure"]!=" "]
result_master = result_master[result_master["Base_Stats"]!=' ']
#itibar sütununda yıldızları silme
result_master["Int_Reputation"]=result_master["Int_Reputation"].str[:-1]

In [14]:
#Sayılabilen Sütunları İntegera çevirme
cols_to_convert=['Age',
 'Overall',
 'Potential',
 'Best_Overall',
 'Growth',
 'Attacking',
 'Crossing',
 'Finishing',
 'Heading_Accuracy',
 'Short_Passing',
 'Volleys',
 'Skill',
 'Dribbling',
 'Curve',
 'FK_Accuracy',
 'Long_Passing',
 'Ball_Control',
 'Movement',
 'Acceleration',
 'Sprint_Speed',
 'Agility',
 'Reactions',
 'Balance',
 'Power',
 'Shot_Power',
 'Jumping',
 'Stamina',
 'Strength',
 'Long_Shots',
 'Mentality',
 'Aggression',
 'Interceptions',
 'Positioning',
 'Vision',
 'Penalties',
 'Composure',
 'Defending',
 'Marking',
 'Standing_Tackle',
 'Sliding_Tackle',
 'Goalkeeping',
 'GK_Diving',
 'GK_Handling',
 'GK_Kicking',
 'GK_Positioning',
 'GK_Reflexes',
 'Total_Stats',
 'Base_Stats',
 'Int_Reputation',
 'PAC',
 'SHO',
 'PAS',
 'DRI',
 'DEF',
 'PHY']

for col in cols_to_convert:
    result_master[col] = result_master[col].astype("int")

In [15]:
result_master.to_pickle("PlayerList2")

In [2]:
player_list= pd.read_pickle("PlayerList2")#enson kullandığımız veriyi çekme

In [3]:
#kullandığımız sütunları çekme
player_list = player_list[['Value','Age','Overall','Potential','Best_Overall',
                           'Position','Growth','Wage','Release_Clause','PAC','SHO','PAS','DRI','DEF','PHY']]
player_list['PositionNum']=player_list.Position.astype("category").cat.codes#mevki string değerlerden oluşmuştur eğitim için integera çevrilmesi gerek 

Y=player_list.loc[:,'Value']#eğitim ve test için verimizi hedef sütunu ve girdi sütunu ayırma 
X=player_list.drop(columns=['Value','Position'])


In [4]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=42)#girdileri test ve eğitim sınıflarına ayırma 
x_test.to_csv(r'x_test.csv',index=False)#dosyaları kaydetmemin sebebi fonksiyonu nasıl iyileştiririm diye çalışmamın kolaylığı için
x_train.to_csv(r'x_train.csv',index=False)
y_test.to_csv(r'y_test.csv',index=False)
y_train.to_csv(r'y_train.csv',index=False)

In [3]:
x_test=pd.read_csv('x_test.csv')
x_train=pd.read_csv('x_train.csv')
y_test=pd.read_csv('y_test.csv')
y_train=pd.read_csv('y_train.csv')

In [4]:
#Girdi sınıflarının katsayıları göre tüm eğitim verisinde çıkan hataların ortalamasını bulan fonksiyon
#cof=katsayı dizisi,x=x_train dizisi,y=y_train dizisi
def ms(cof, x, y):#Multiple Linear Fonksiyonum
    return np.mean((np.dot(x, cof) - y)**2)/2

#katsayıyı güncellemek için kullanılan fonksiyon ,doğruya yaklaştırıyor ,eğimi döndürüyor.
def gradients(cof, x, y):
    return np.mean(x.transpose()*(np.dot(x, cof) - y), axis = 1)

#Multilinear_regression katsayı dizisini oluştuğu yer,ilk başta katsasyı dizi=0
#lr = bizim öğrenme katsayımız,b1 cof dizisini derecesi için değişme katsasyımız,
# b2:cof değerinini vektörel olarak değişmesini sağlayan katsayımız,
# epsilon son hata değeri ile bir önceki değerin değişimi ayrıca eğitimin tamamlanması için verilen katsayı
#başta verilen katsayılara göre doğruluk skoru değişebilir veya hıza değişebilir
def multilinear_regression(cof, x, y, lr, b1 = 0.9, b2 = 0.999, epsilon = 1e-8):
    prev_error = 0
    m_cof = np.zeros(cof.shape)
    v_cof = np.zeros(cof.shape)
    moment_m_cof = np.zeros(cof.shape)
    moment_v_cof = np.zeros(cof.shape)
    t = 0
    while True:
        #hata buluyoruz
        error = ms(cof, x, y)
        #son hata değerimizle bir önceki hata değerinin farkı epsilondan küçük olana kadar
        if abs(error - prev_error) <= epsilon:
            break
        prev_error = error
         #ilk başta verilen katsayının x,y için eğimini buluyoruz
        grad = gradients(cof, x, y)
        t += 1
        #derece eğimini güncelledik,değişkenler ile
        m_cof = b1 * m_cof + (1-b1)*grad
        #eğimi vektörünü güncelledik
        v_cof = b2 * v_cof + (1-b2)*grad**2
        moment_m_cof = m_cof / (1-b1**t)
        moment_v_cof = v_cof / (1-b2**t)
          #Güncel katsayı dizisini güncellemek için vektörel ve scaler büyükleri çarpıp bir değişim oranı bulma
        delta = ((lr / moment_v_cof**0.5 + 1e-8) *
                 (b1 * moment_m_cof + (1-b1)*grad/(1-b1**t)))
        # yeni katsayı değerleri hesapladık
        cof = np.subtract(cof, delta)
    return cof
#0 ile başlayan katsayı dizisi
cof = np.zeros(14)
#Tek sütunlu dataframeleri işlem yapmak için series çevirdik
y_train = pd.Series(y_train['Value'], index=y_train.index)
y_test = pd.Series(y_test['Value'], index=y_test.index)
c = multilinear_regression(cof, x_train, y_train, 1e-1)

In [5]:
c.to_pickle("modelkatsayı")

In [7]:
def tahmin(coef,deneme):#oluşan son ağırlığa göre bir tane girişi tahmin etme,çarpım sonucu bir dizi buluyoruz,elemanlarını toplamı döndürme
    tahmin1= coef.mul(deneme)
    a=0
    a=tahmin1['Age']+tahmin1['Overall']+tahmin1['Potential']+tahmin1['Best_Overall']+tahmin1['Growth']
    a=a+tahmin1['PAC']+tahmin1['SHO']+tahmin1['PAS']+tahmin1['DRI']+tahmin1['DEF']+tahmin1['PHY']              
    a=a+tahmin1['Release_Clause']+ tahmin1['Wage']+ tahmin1['PositionNum']
    return a


In [8]:
#Eğitimin başarısını ölçmek için R2_score fonksiyonunu kullandık.
#R2 algoritmasın da 2 tür hata vardır,birincisi tahminden ile gerçek sonuç değerini çıkarıyoruz ve karesini alıyoruz (tüm indexler)  
#2.hatası ise ,tahmin ile gercek sonuç ortalamasını çıkartıyoruz.En sonunda ise 1-(ERROR1/ERROR2) değeri skor oluyor.
def r2score(x_test,y_test,model):#r2 score hesaplanması
    for i in range(0,len(x_test.index)):
        actual=y_test.iloc[i]
        actual_mean=y_test.mean()
        deneme = x_test.iloc[i]
        predicted=tahmin(model,deneme)
        error1=predicted-actual
        error1=error1**2
        error2=predicted-actual_mean
        error2=error2**2
        score=1-(error1/error2)
    return score
score=r2score(x_test,y_test,c)

In [40]:
form=tk.Tk()
form.title('Futbolcu Piyasa Değeri Tahminleme')
form.geometry('720x500+300+100')

def predict():
    giris= pd.DataFrame(columns=['Age','Overall','Potential','Best_Overall'
                        ,'Growth','Wage','Release_Clause','PAC','SHO','PAS','DRI','DEF','PHY','PositionNum'])
    thisdict = {"CB": 0, "CAM": 1, "CDM": 2, "CF": 3, "CM": 4, "GK": 5, "LB": 6,
    "LM": 7, "LW": 8, "LWB": 9, "RB": 10, "RM": 11, "RW": 12, "RWB": 13, "ST": 14}
    positionnumx = thisdict[position_e.get()]

    giris=giris.append({'Age': int(age_e.get()), 'Overall': int(overall_e.get()), 'Potential':int(potential_e.get()),'Best_Overall':int(b_overall_e.get()),'Growth':int(growth_e.get()),'Wage':float(wage_e.get()),'Release_Clause':int(rc_e.get()),'PAC':int(pace_e.get()),'SHO':int(sho_e.get()),'PAS':int(pass_e.get()), 'DRI':int(dri_e.get()),'DEF':int(deff_e.get()),'PHY':int(phy_e.get()),'PositionNum':positionnumx},ignore_index=True)
    giris=giris.iloc[0]
    
    pred = tahmin(c,giris)
    sonuc=tk.Label(text='Futbolcunun Değeri: '+ str(pred) ,font=50, background='Light green')
    sonuc.place(x=90,y=200)

age_l=tk.Label(form, text='Age')
age_l.place(x=10,y=30)
overall_l=tk.Label(text='Overall')
overall_l.place(x=10,y=60)
potential_l=tk.Label(form, text='Potential')
potential_l.place(x=10,y=90)
b_overall_l=tk.Label(text='Best Overall')
b_overall_l.place(x=10,y=120)
position_l=tk.Label(form, text='Position')
position_l.place(x=10,y=150)
growth_l=tk.Label(text='Growth')
growth_l.place(x=250,y=30)
wage_l=tk.Label(form, text='Wage')
wage_l.place(x=250,y=60)
rc_l=tk.Label(text='Release Clause')
rc_l.place(x=250,y=90)
pace_l=tk.Label(form, text='Pace')
pace_l.place(x=250,y=120)
sho_l=tk.Label(text='Shooting')
sho_l.place(x=250,y=150)
pass_l=tk.Label(form, text='Passing')
pass_l.place(x=480,y=90)
dri_l=tk.Label(text='Dribbling')
dri_l.place(x=480,y=120)
deff_l=tk.Label(form, text='Defending')
deff_l.place(x=480,y=30)
phy_l=tk.Label(text='Physical')
phy_l.place(x=480,y=60)


age_e=tk.Entry()
age_e.place(x=90,y=30)
overall_e=tk.Entry()
overall_e.place(x=90,y=60)
potential_e=tk.Entry()
potential_e.place(x=90,y=90)
b_overall_e=tk.Entry()
b_overall_e.place(x=90,y=120)
position_e=tk.Entry()
position_e.place(x=90,y=150)
growth_e=tk.Entry()
growth_e.place(x=340,y=30)
wage_e=tk.Entry()
wage_e.place(x=340,y=60)
rc_e=tk.Entry()
rc_e.place(x=340,y=90)
pace_e=tk.Entry()
pace_e.place(x=340,y=120)
sho_e=tk.Entry()
sho_e.place(x=340,y=150)
pass_e=tk.Entry()
pass_e.place(x=560,y=30)
dri_e=tk.Entry()
dri_e.place(x=560,y=60)
deff_e=tk.Entry()
deff_e.place(x=560,y=90)
phy_e=tk.Entry()
phy_e.place(x=560,y=120)

button=tk.Button(form,text='HESAPLA',command=predict)
button.place(x=10,y=200)


form.mainloop()

GK
20
    Age  Overall  Potential  Best_Overall  Growth  Wage  Release_Clause   PAC  \
0  20.0     80.0       90.0          85.0    10.0   0.5            50.0  70.0   

    SHO   PAS   DRI   DEF   PHY  PositionNum  
0  70.0  70.0  70.0  70.0  70.0          5.0  
